In [14]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

import argparse
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch_sparse import SparseTensor
from torch_geometric.utils import to_undirected, dropout_adj
import os
import os.path as osp

from ogb.nodeproppred import PygNodePropPredDataset

import torch
from torch_geometric.data import Data

import os.path as osp

## Загрузка данных

In [37]:
data = pd.read_csv('./Data/GNN_final_data.csv')

In [38]:
data.shape

(7015937, 14)

In [39]:
data

,date_key,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence,Reject_presence
0,2018-03-01,41000.00,1279230,1,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
1,2018-03-01,618280.00,1279230,1003953,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
2,2018-03-01,2698540.00,1279230,682959,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
3,2018-03-01,3000.00,1279230,1852820,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
4,2018-03-01,1000.00,1279230,119556,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7015932,2021-03-01,1000.00,673145,1003953,47.0,127.0,0,0,1,0,0,1,0.0,0.0
7015933,2021-03-01,290.00,764479,119556,46.0,375.0,0,0,1,0,0,1,0.0,1.0
7015934,2021-03-01,2000.00,398315,119556,34.0,126.0,0,1,0,0,0,1,0.0,0.0
7015935,2021-03-01,10855.68,1095817,1095817,42.0,187.0,0,0,1,0,1,0,0.0,0.0


In [40]:
unique_inn = data.client_inn.unique()

In [41]:
unique_inn

array([1279230, 1224261, 1549773, ...,  398315, 1095817,  136009])

In [42]:
unique_inn.shape

(63864,)

# Нужно ли убирать в данных для x client_inn и partner_inn???

In [43]:
data_with_inn = data

In [44]:
#data = data.drop(['client_inn', 'partner_inn'], axis=1)

In [45]:
data

,date_key,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence,Reject_presence
0,2018-03-01,41000.00,1279230,1,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
1,2018-03-01,618280.00,1279230,1003953,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
2,2018-03-01,2698540.00,1279230,682959,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
3,2018-03-01,3000.00,1279230,1852820,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
4,2018-03-01,1000.00,1279230,119556,54.0,6251.0,1,0,0,0,0,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7015932,2021-03-01,1000.00,673145,1003953,47.0,127.0,0,0,1,0,0,1,0.0,0.0
7015933,2021-03-01,290.00,764479,119556,46.0,375.0,0,0,1,0,0,1,0.0,1.0
7015934,2021-03-01,2000.00,398315,119556,34.0,126.0,0,1,0,0,0,1,0.0,0.0
7015935,2021-03-01,10855.68,1095817,1095817,42.0,187.0,0,0,1,0,1,0,0.0,0.0


In [46]:
data.shape

(7015937, 14)

In [47]:
#data_with_inn.drop_duplicates(subset=['client_inn'])

## Перевод исходного датасета в датасет для графов

In [48]:
features_dataset = data.drop(['Fraud_presence', 'date_key'], axis=1)

In [49]:
features_dataset

,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Reject_presence
0,41000.00,1279230,1,54.0,6251.0,1,0,0,0,0,1,0.0
1,618280.00,1279230,1003953,54.0,6251.0,1,0,0,0,0,1,0.0
2,2698540.00,1279230,682959,54.0,6251.0,1,0,0,0,0,1,0.0
3,3000.00,1279230,1852820,54.0,6251.0,1,0,0,0,0,1,0.0
4,1000.00,1279230,119556,54.0,6251.0,1,0,0,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7015932,1000.00,673145,1003953,47.0,127.0,0,0,1,0,0,1,0.0
7015933,290.00,764479,119556,46.0,375.0,0,0,1,0,0,1,1.0
7015934,2000.00,398315,119556,34.0,126.0,0,1,0,0,0,1,0.0
7015935,10855.68,1095817,1095817,42.0,187.0,0,0,1,0,1,0,0.0


In [50]:
features_dataset.shape

(7015937, 12)

In [56]:
x = torch.tensor(features_dataset.drop(['client_inn', 'partner_inn'], axis=1).values.astype(np.float32)) 

In [57]:
y = torch.tensor(data['Fraud_presence'].values.astype(np.float32))

In [58]:
#Получили number_inn при помощи labelencoder
from sklearn import preprocessing
labelencoder = preprocessing.LabelEncoder()

#data_with_inn['number_inn'] = labelencoder.fit_transform(data_with_inn['client_inn', 'partner_inn'])
#data_with_inn

features_dataset['client_inn_Cat'] = labelencoder.fit_transform(features_dataset['client_inn'])
features_dataset

features_dataset['partner_inn_Cat'] = labelencoder.fit_transform(features_dataset['partner_inn'])
features_dataset

,amount,client_inn,partner_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Reject_presence,client_inn_Cat,partner_inn_Cat
0,41000.00,1279230,1,54.0,6251.0,1,0,0,0,0,1,0.0,43865,0
1,618280.00,1279230,1003953,54.0,6251.0,1,0,0,0,0,1,0.0,43865,801445
2,2698540.00,1279230,682959,54.0,6251.0,1,0,0,0,0,1,0.0,43865,546297
3,3000.00,1279230,1852820,54.0,6251.0,1,0,0,0,0,1,0.0,43865,1469984
4,1000.00,1279230,119556,54.0,6251.0,1,0,0,0,0,1,0.0,43865,94719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7015932,1000.00,673145,1003953,47.0,127.0,0,0,1,0,0,1,0.0,26875,801445
7015933,290.00,764479,119556,46.0,375.0,0,0,1,0,0,1,1.0,30057,94719
7015934,2000.00,398315,119556,34.0,126.0,0,1,0,0,0,1,0.0,14456,94719
7015935,10855.68,1095817,1095817,42.0,187.0,0,0,1,0,1,0,0.0,40038,875714


In [59]:
edge_index = features_dataset[['client_inn_Cat', 'partner_inn_Cat']].values.T
edge_index = torch.tensor(edge_index).to(dtype=torch.long)

In [60]:
edge_index

tensor([[ 43865,  43865,  43865,  ...,  14456,  40038,   5350],
        [     0, 801445, 546297,  ...,  94719, 875714,  94719]])

In [61]:
#Перевод данных в датасет для графа
data_graph = Data(x=x, y=y, edge_index=edge_index)

In [62]:
data_graph

Data(x=[7015937, 12], edge_index=[2, 7015937], y=[7015937])

In [63]:
x = data_graph.x
N = data_graph.num_nodes

In [64]:
print(N)

7015937


In [65]:
data_graph.edge_index

tensor([[ 43865,  43865,  43865,  ...,  14456,  40038,   5350],
        [     0, 801445, 546297,  ...,  94719, 875714,  94719]])

In [66]:
data_graph.edge_index.shape

torch.Size([2, 7015937])

In [67]:
path = './adj_gcn.pt'
print('Making the graph undirected.')

data_graph.edge_index, _ = dropout_adj(
       data_graph.edge_index, p=0, num_nodes=data_graph.num_nodes)

data_graph.edge_index

Making the graph undirected.


/data/home/krazheva/.conda/envs/py310/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


tensor([[ 43865,  43865,  43865,  ...,  14456,  40038,   5350],
        [     0, 801445, 546297,  ...,  94719, 875714,  94719]])

In [68]:
data_graph

Data(x=[7015937, 12], edge_index=[2, 7015937], y=[7015937])

In [69]:
data_graph.num_nodes

7015937

In [70]:
data_graph.edge_index.shape

torch.Size([2, 7015937])

In [71]:
data_graph.edge_index = to_undirected(data_graph.edge_index, num_nodes=data_graph.num_nodes)

In [72]:
print(data_graph.edge_index)

tensor([[      0,       0,       0,  ..., 1524619, 1524620, 1524621],
        [      0,       2,       3,  ...,   48298,   42884,   50087]])


In [73]:
type(data_graph.edge_index)

torch.Tensor

In [74]:
torch.max(data_graph.edge_index)

tensor(1524621)

In [75]:
row, col = data_graph.edge_index
print('Computing adj...')

adj = SparseTensor(row=row, col=col, sparse_sizes=(N, N))
adj = adj.set_diag()
deg = adj.sum(dim=1).to(torch.float)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
adj = deg_inv_sqrt.view(-1, 1) * adj * deg_inv_sqrt.view(1, -1)
adj = adj.to_scipy(layout='csr')

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)
adj=sparse_mx_to_torch_sparse_tensor(adj)
print('Start processing')

Computing adj...
Start processing


In [ ]:
num_hops = 3
#saved = torch.cat((x[train_idx], x[valid_idx], x[test_idx]), dim=0)
torch.save(data_graph, f'./graph1.pt')

for i in tqdm(range(num_hops)):
    x = adj @ x
    torch.save(x, f'./graph1_hope_{i+1}.pt')

 67%|██████▋   | 2/3 [00:06<00:03,  3.23s/it]